# Overview

The following example demonstrates a data science workflow using real world, publicly available monthly data on crimes against senior citizens collected by law enforcement agencies in California. The CSV data file used for this example can be obtained from https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2022-08/VCASC2000-2021.csv.

This example focuses on the process of reshaping a data set from long to wide format.

# Load the SWAT Library and Connect to the CAS Server

Load the SWAT library and then create a connection to the CAS server using the CAS function and assign the CAS connection object to the variable s. The first argument specifies the host name and the second argument specifies the port.

In [ ]:
import swat
# change the host and port to match your site
s = swat.CAS("cloud.example.com", 10065)

# Load the Data

## Load the Data from a Caslib

The default method of loading data is to load the data from the data source portion of a caslib, which is known as a server-side load. This requires the data file to be saved in the active caslib (Casuser). Once the file has been saved to the caslib, use the table.loadTable action to load the VCASC2000-2021.csv file from the data source portion of the Caslib into memory as a CAS table.

In the casOut parameter, specify that the data is saved as a table named vcasc.
In the importOptions parameter, specify CSV for the fileType.
For encoding, specify latin1.
For guessRows, specify a number that includes all rows in the data set, such as 200,000 so that all rows will be scanned to determine the appropriate data type for each column.

In [ ]:
s.table.loadTable(path="VCASC2000-2021.csv",
                  caslib="casuser",
                  casOut={"name":"vcasc", 
                          "replace":True},
                  importOptions={"fileType":"CSV",
                                 "encoding":"latin1",
                                 "guessrows":200000})

## Load a Client-Side File into CAS

Another method of loading data into CAS memory is to load the data from an external source that is accessible to the CAS server. This example uses the upload_file function to perform a client-side load.

In [ ]:
s.read_csv("https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2022-08/VCASC2000-2021.csv", 
           encoding='latin1',
           casout={"name":"vcasc", 
                   "caslib":"casuser",
                   "replace":True})  

# Explore the Data

Examine the rows and columns in the data.

## Reference the In-Memory Table

In [ ]:
tbl = s.CASTable(name='vcasc', caslib='casuser')

## Examine the Rows

The table.fetch action can be used to retrieve a specified number of rows from the data. Here, the parameter n=20 is specified, which indicates that the first 20 rows are retrieved. The fetch action shows that the data is presented in a long format, where monthly crime counts reported by each agency (NCIC_AGENCY_CODE) are distributed across multiple rows rather than multiple columns. In other words, for each unique reporting agency, there is a separate row for each month of the year with the counts of each crime type reported for that month.

In [ ]:
tbl.fetch(to=20)

## Examine the Columns

Use the table.columnInfo action to obtain metadata about the table. The result includes the names of columns, and information about each column, including its label (if applicable), type, length, and format. The results show that the columns are the appropriate types.

In [ ]:
tbl.columnInfo()

## Identify Unique Values of the Year Column

Examine the YEAR column using a simple.freq to create a frequency distribution showing the unique values of year. The result shows that data are available for the years 2000 to 2021.

In [ ]:
tbl.freq(inputs={"Year"})

# Prepare and Analyze the Data

## Filter Rows

This example will use data from 2021 only. Therefore, data for all other years should be filtered out. The table.deleteRows action can be used to remove rows from a table. The where subparameter specifies that any rows satisfying the condition will be deleted.

In [ ]:
tbl.where = "YEAR NE 2021"
tbl.deleteRows()

## Reshape the Data from Long to Wide Format

The data are currently in a long format with many rows and few columns. The data can be reshaped into a wide format with few rows and many columns. In the wide format, information about each observation (for example, number of crimes in each month) is stored in separate columns, rather than rows. The wide format shows only one row for each unique value of the key column that uniquely identifies observations. Certain statistical techniques might require data to be in the wide format.

The dataShaping.longToWide action is used to reshape a table from long to wide. Here, the long table vcasc is reshaped into a wide table named vcascWide.

For the groupBy subparameter, specify the column NCIC_AGENCY_CODE as the column used to define the groups and whose values will be collapsed so that the reshaped data contains a single row for each unique value of the column.

For the optional groupByMode subparameter, specify REDISTRIBUTE to guarantee ordering within groups.

For the id parameter, specify MONTH as the column whose unique values will be used to define the columns in the reshaped table.

For the inputs parameter, specify a column containing statistics, such as SUB_ROBBERY, as the column whose values will be the values of the newly created columns in the reshaped wide table.

Because the month column specified in the id parameter contains 12 unique values that are used to create the corresponding columns in the wide data, specify a value of 12 for the maxPosition parameter.

Include the parameters sum and mean and specify SUB_ROBBERY as the argument to calculate these statistics for each observation across all 12 month columns in the reshaped data.

In [ ]:
del tbl.where

s.builtins.loadActionSet("dataShaping")            

s.dataShaping.longToWide(                        
      table={"name":"vcasc",                      
             "groupBy":"NCIC_AGENCY_CODE",
             "groupByMode":"REDISTRIBUTE"},
      inputs={"SUB_ROBBERY"},      
      sum={"SUB_ROBBERY"},
      mean={"SUB_ROBBERY"},
      maxPosition=12,
      id={"MONTH"},                                                             
      casout={"name":"vcascWide", 
              "replace":True})

tbl_wide = s.CASTable("vcascWide", caslib="casuser")

Use the table.fetch action to view the first 10 rows of the reshaped table vcascWide. The reshaped table now shows only a single row for each unique value of NCIC_AGENCY_CODE. Each of the unique values of the MONTH column defines the column names (for example, month 1 is SUB_ROBBERY_0, month 2 is SUB_ROBBERY_1, and so on).The values of the SUB_ROBBERY column are stored in the columns that correspond to the unique values of the MONTH column.

In [ ]:
tbl_wide.fetch(to=10)

Examine the data to confirm that there is now only 1 row for each distinct value. Use the simple.distinct action to count the distinct values of the groupBy column (NCIC_AGENCY_CODE).

In [ ]:
tbl_wide.distinct(inputs=[{"name":"NCIC_AGENCY_CODE"}])

Use the table.recordCount action to count the number of rows in the data. The resulting count matches the number of distinct values of NCIC_AGENCY_CODE.

In [ ]:
tbl_wide.recordCount()

## Clean the Reshaped Table

Use the table.alterTable action to modify the reshaped table to make it easier to read by removing undeeded columns (MONTH and _Frequency_) and renaming the columns containing computed statistics and the monthly counts to their corresponding months. The MONTH and the _Frequency_ columns were created during the reshaping process but contain unnecessary information and can be dropped from the wide table.

In [ ]:
tbl_wide.alterTable(columns=[{"name":"SUB_ROBBERY_0", "rename":"Jan"},
                             {"name":"SUB_ROBBERY_1", "rename":"Feb"},
                             {"name":"SUB_ROBBERY_2", "rename":"Mar"},
                             {"name":"SUB_ROBBERY_3", "rename":"Apr"},
                             {"name":"SUB_ROBBERY_4", "rename":"May"},
                             {"name":"SUB_ROBBERY_5", "rename":"Jun"},
                             {"name":"SUB_ROBBERY_6", "rename":"Jul"},
                             {"name":"SUB_ROBBERY_7", "rename":"Aug"},
                             {"name":"SUB_ROBBERY_8", "rename":"Sep"},
                             {"name":"SUB_ROBBERY_9", "rename":"Oct"},
                             {"name":"SUB_ROBBERY_10", "rename":"Nov"},
                             {"name":"SUB_ROBBERY_11", "rename":"Dec"},
                             {"name":"SUB_ROBBERY_sum", "rename":"Total"},
                             {"name":"SUB_ROBBERY_mean", "rename":"Average"},
                    ],
                    drop=["MONTH", "_Frequency_"])         

Use a second alterTable action to reorder the NCIC_AGENCY_CODE column so that it is the first column in the data.

In [ ]:
tbl_wide.alterTable(columnOrder=["NCIC_AGENCY_CODE"])

Use a table.fetch action to view the cleaned table. The table now shows the NCIC_AGENCY_CODE column first and each of the reshaped monthly count columns has been renamed according to the month that it represents. The table also includes the columns Average and Total containing the calculated statistics.

In [ ]:
tbl_wide.fetch(index=False)

## Add a New Column

The table.copyTable action with the computedVarsProgram subparameter can be used to calculate or add new columns in a table. Using the vcascWide table, create a new column to indicate that the counts are specifically for robberies and name the output table vcascWideType.

In [ ]:
tbl_wide.computedVars = {"name":"TYPE"}
tbl_wide.computedVarsProgram = "TYPE='Robbery'"

tbl_wide.copyTable(casOut={"caslib":"casuser",          
                          "name":"vcascWideType", 
                          "replace":True})

tbl_widetype = s.CASTable("vcascWideType", caslib="casuser")

## View the Reshaped Table Sorted by Total Column

Show the top 10 reporting agencies with the highest total number of robberies. Use a table.fetch action and sort by the total column in descending order.

In [ ]:
tbl_widetype.fetch(to=10, 
                   sortBy=[{"name":"TOTAL", "order":"DESCENDING"}])